In [2]:
import pandas as pd
import pandasdmx 
import requests as rq
import re

In [2]:
#from mt_stuff import dict_OECD_indicators

In [3]:
economic_activities = [
    'PIEAMP02',
    'PIEAMP01',
    'PIEATI02',
    'PIEATI01',
    'PIEAMI02',
    'PIEAMI01',
    'PIEAFD02',
    'PIEAFD01',
    'PIEAEN02',
    'PIEAEN01',
]

# 'PIEAMP02': 'Economic activities - Domestic producer prices - Manufacturing',
# 'PIEAMP01': 'Economic activities - Total producer prices - Manufacturing',
# 'PIEATI02': 'Economic activities - Domestic producer prices - Industrial Activities',
# 'PIEATI01': 'Economic activities - Total producer prices - Industrial Activities',
# 'PIEAMI02': 'Economic activities - Domestic Producer prices - Mining and quarrying activities',
# 'PIEAMI01': 'Economic activities - Total Producer prices - Mining and quarrying activities',
# 'PIEAFD02': 'Economic activities - Domestic Producer prices - Manufacture of food products',
# 'PIEAFD01': 'Economic activities - Total Producer prices - Manufacture of food products',
# 'PIEAEN02': 'Economic activities - Domestic Producer prices - Energy',
# 'PIEAEN01': 'Economic activities - Total Producer prices - Energy',

In [4]:
dict_OECD_indicators = {

    # B6BLTT01 - current account balance
    'Balance_of_payments': {
        'dbname':  "MEI_BOP6",
        'query': ["B6BLTT01", "AUS", "CXCUSA", "Q"]
        
    },

    'Business_tendency_and_consumer_opinion_surveys': {
        'dbname': "MEI_BTS_COS",
        # SUBJECT', 'LOCATION', 'MEASURE', 'FREQUENCY', 'TIME_PERIOD'
        'query': ["BSCI", "AUS", "BLSA", "Q"]
    },
    'Composite_leading_indicators':
    {
        'dbname': "MEI_CLI",
        # CSCICP03 OECD Standardised CCI, Consumner confidenceAmplitude adjusted (Long term average=100), sa"
        # LORSGPOR_IXOBSA "Original, seasonally adjusted (GDP)"
        # LOLITONO name: "Normalised (CLI) name: "Composite Leading Indicator (CLI)"
        #     ('LORSGPTD', 'ZAF', 'M', '2020-11')],
        # 'BSCICP03': 'OECD Standardised BCI, Amplitude adjusted (Long term average=100), sa',
        # ['SUBJECT', 'LOCATION', 'FREQUENCY', 'TIME_PERIOD']
        # cli_indicators=['CSCICP03','LORSGPOR_IXOBSA','LOLITONO','BSCICP03']
        # only one is quaterly
        # cli_series.loc[cli_indicators,"AUS","Q"]
        'query': ['LORSGPOR_IXOBSA', 'AUS', "Q"]
    },


    'Financial_statistics': {
        'dbname': "MEI_FIN",

        # 'SP': 'Share Prices, Index',
        # Look at exchange rates if time permits
        
        'query': [["SP", "AUS", 'Q'], ["IRLT", "AUS", 'Q']],
        'names' : ['share_price_index', 'Long-term interest rates']
        # Long-term interest rates, Per cent per annum"
        # mei_series.loc["IRLT","AUS",'Q']
    },
    'Industry': {
        'dbname': "MEI_REAL",
        # 'SUBJECT', 'LOCATION', 'FREQUENCY', 'TIME_PERIOD']
        # 'Production of total industry sa, Index'
        'query': ["PRINTO01", 'AUS', 'Q']
    },
    'International_trade': {
        'dbname': "MEI_TRD",
        # measure CXMLSA "US-Dollar converted, Seasonally adjusted" Measure
        'query': ["XTNTVA01", 'AUS', "CXMLSA", "Q"]
    },
    'Labour_market_statistics': {
        'dbname': "STLABOUR",
        # names=['LOCATION', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME_PERIOD']
        'query': ["AUS", 'LREM64TT', 'STSA', 'Q']
    },


    'Consumer_price_indices': {
        'dbname': "PRICES_CPI",
        # CPALTT01
        #['LOCATION', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME_PERIOD']
        # id: "GP" name: "Percentage change from previous period"
        'query': ['AUS', 'CPALTT01', 'GP', 'Q'],
    },
   # 'Producer_price_indices': {
   #     'dbname': "MEI_PRICES_PPI",
   #     # measure available GP - percentatge change from previous perios
   #     # ['SUBJECT', 'LOCATION', 'MEASURE', 'FREQUENCY', 'TIME_PERIOD'
   #     'query': [economic_activities, 'AUS', 'GP', 'Q']
   # },
    'Purchasing_power_parities': {
        'dbname': "PPPGDP",
        #['INDIC', 'LOCATION', 'TIME_PERIOD']
        # Purchasing Power Parities for GDP PPPGDP
        'query': [["PPP", "AUS"], ["CPL", "AUS"]],
        'names' : ['Purchasing_Power_Parities', 'Comparative_Price_Levels']
        # 'CPL': 'Comparative Price Levels',

    },
  #  'Comparative_price_level': {
  #          # COMPARISON METRIC NOT REQUIRED
  #          'dbname':   "CPL",
  #          # these are monthly values
  #          # PRINTO01
  #          # ['INDIC', 'LOCATION', 'CPL_COUNTRY', 'TIME_PERIOD'], length=1444)
  #          'query': ["CPL", "AUS", "USA"]
  # },
    'Quarterly_national_accounts': {
            'dbname': "QNA",
            # dict_values["PRINTO01"]

            # ['LOCATION', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME_PERIOD'
            # 'US dollars, volume estimates, fixed PPPs, OECD reference year, annual levels, seasonally adjusted
            'query': ["AUS", 'B1_GE', "VOBARSA", "Q"]
    },
}

In [3]:
#stru = getOECDStructure(dsname="MEI_BOP6")

In [5]:
def getOECDStructure(dsname, root_dir = 'http://stats.oecd.org/SDMX-JSON/dataflow'):
    
    """
    Check structure of OECD dataset.
    
    Parameters
    -----
    dsname: str
        dataset identifier (e.g., MEI for main economic indicators)
    root_dir: str
        default OECD API structure uri
    showValues: list
        shows available values of specified variable, accepts list of integers
        which mark position of variable of interest (e.g. 0 for LOCATION)
          
    Returns
    -----
    results: list
        list of dictionaries with observations parsed from JSON object, 
        
    """ 
    
    url = root_dir + '/' + dsname + '/all'
    
    print('Requesting URL ' + url)
    
    response = rq.get(url = url)
    
    if (response.status_code == 200):
        
        responseJson = response.json()
        
        return responseJson
        
    else:
        
        print('\nError: %s' % response.status_code)

In [7]:
oecd = pandasdmx.Request('OECD')

In [9]:
# TODO: check for other data where not required
quarters = [ f'-Q{i}' for i in range(1,5)]
quarters

['-Q1', '-Q2', '-Q3', '-Q4']

In [11]:
country ='AUS' # Country-CODE # UK , USA 
freq='Q'
startYear = 2000 
series_list = []
for k, v in dict_OECD_indicators.items():
    dbname = v['dbname']
    query_def = v['query']
    name = k
    #print(f"query_def -->{query_def}")
    data_response = oecd.data(resource_id=dbname, key=f'all?startTime={startYear}')
    data_series= data_response.to_pandas()
    if dbname == "MEI_PRICES_PPI":
        ## this may not behave well as different countries publish different indicators
        print(f"All countries not reporting same metrics for -->{dbmane} -- Producer Prices Not used in computation")
        #indicators = query_def[0]
        #print(f"Indicators -->{indicators}")
        #print()
        #for indi in indicators:
        #    query_def[0] = indi
        #    query= [str(item).replace('AUS', country) for item in query_def]
        #    query= [str(item).replace('Q', freq) for item in query]
        #    print(f" {query} -->")
        #    series_data = data_series.loc[tuple(query)]
        #    series_list.append(series_data)

    elif type(query_def[0]) == list:
        for arr in query_def:
            index = query_def.index(arr)
            col_name = f'{name}_{v["names"][index]}'
            query= [str(item).replace('AUS', country) for item in arr]
            query= [str(item).replace('Q', freq) for item in query]
            print(f" {query} -->")
            series_data = data_series.loc[tuple(query)]
            if   dbname == 'PPPGDP' :
                # change to quarters from annual
                dict_new_series= {}
                for t in series_data.index:
                    for q in quarters:
                        new_index = [t + q for q in quarters]
                        dict_new_series[t+q]= (series_data.loc[t] / 4 ) 
                series_data = pd.Series(dict_new_series)
                series_data.name = col_name

            else: 
                series_data.name = col_name
                series_list.append(series_data)            
    

    else:
        query= [str(item).replace('AUS', country) for item in query_def]
        query= [str(item).replace('Q', freq) for item in query]
        #query= [w.replace('Q', freq) for w in query]
        #print(f'{len(query)} ---> {query}')
        #print(query)
        series_data= data_series.loc[tuple(query)]
        series_data.name = name
        series_list.append(series_data)
        #print(series_test)


    #id is the indicator name 
    #for key in val:
        #dbname = val[dbname]
     #print(f'Key : {key} --> Val {val[key]}')
     #One get the data
     #data = oecd.data(resource_id=dsname, key=f'all?startTime={startYear}')
     #Analyse the data

 ['SP', 'AUS', 'Q'] -->
 ['IRLT', 'AUS', 'Q'] -->
 ['PPP', 'AUS'] -->
 ['CPL', 'AUS'] -->


In [13]:
df_all_indicators = pd.concat(series_list,axis=1)

In [14]:
df_all_indicators

,Balance_of_payments,Business_tendency_and_consumer_opinion_surveys,Composite_leading_indicators,Financial_statistics_share_price_index,Financial_statistics_Long-term interest rates,Industry,International_trade,Labour_market_statistics,Consumer_price_indices,Quarterly_national_accounts
TIME_PERIOD,,,,,,,,,,
2000-Q1,-5137.081,12.934080,64.16375,57.14708,6.901983,71.74657,-1.609661,68.51615,0.868307,1052610.8
2000-Q2,-4915.593,7.188005,64.75610,56.61853,6.300913,72.05145,-1.704450,69.11356,0.717360,1062328.3
2000-Q3,-3290.672,8.206771,64.88159,60.17802,6.175162,73.01106,-0.838502,69.65662,3.846154,1064387.1
2000-Q4,-3131.326,-10.157550,64.63844,59.63087,5.880219,73.32154,0.025222,69.05394,0.274348,1060398.0
2001-Q1,-2248.219,-6.817195,65.29521,59.79416,5.285803,73.00371,0.459063,68.90128,1.094391,1071172.5
...,...,...,...,...,...,...,...,...,...,...
2020-Q1,4550.063,-1.000000,110.87390,117.48830,1.006667,111.62420,13.219240,74.48933,0.344234,1818611.6
2020-Q2,10804.130,-14.000000,103.10910,101.68970,0.896667,108.57980,12.807730,70.61123,-1.886792,1691897.7
2020-Q3,7560.197,2.333333,106.61850,109.36950,0.890000,108.65250,9.760626,72.08835,1.573427,1750397.7


In [17]:
df_all_indicators.to_csv('./data/AUS_2000.csv')

In [ ]:
df.rename

In [16]:
df_all_indicators.rename(columns={"Quarterly_national_accounts": "GDP"},inplace=True)

In [139]:
df_all_indicators

,Balance_of_payments,Business_tendency_and_consumer_opinion_surveys,Composite_leading_indicators,Financial_statistics_share_price_index,Financial_statistics_Long-term interest rates,Industry,International_trade,Labour_market_statistics,Consumer_price_indices,GDP
TIME_PERIOD,,,,,,,,,,
2019-Q1,-1661.208,15.333330,109.3422,109.5613,2.120000,109.0664,11.402610,74.06125,0.000000,1792868.1
2019-Q2,3306.409,22.666670,110.0532,116.4485,1.630000,109.9773,15.836640,74.27474,0.613497,1804490.3
2019-Q3,5543.365,7.000000,110.7155,120.9353,1.096667,111.1706,17.623950,74.47119,0.522648,1816941.8
2019-Q4,2742.469,20.666670,111.2023,122.3597,1.126667,111.7893,12.564330,74.33272,0.693241,1823929.1
2020-Q1,4550.063,-1.000000,110.8739,117.4883,1.006667,111.8988,13.219240,74.48933,0.344234,1818611.6
2020-Q2,10804.130,-14.000000,103.1091,101.6897,0.896667,108.5698,12.807730,70.61123,-1.886792,1691897.7
2020-Q3,7560.197,2.333333,106.6185,109.3695,0.890000,108.8017,9.760626,72.08835,1.573427,1750397.7
2020-Q4,11689.750,20.333330,109.9532,116.2053,0.890000,108.4331,11.988090,73.47577,0.860585,1806519.7
2021-Q1,14131.750,21.000000,NaN,123.1831,1.353333,NaN,17.679190,74.34683,0.597270,1838799.1


In [140]:
df_all_indicators.to_csv('./data/2019.csv')

In [81]:
for ser in series_list:
    print(ser.name)

Balance_of_payments
Business_tendency_and_consumer_opinion_surveys
Composite_leading_indicators
Financial_statistics_share_price_index
Financial_statistics_Long-term interest rates
Industry
International_trade
Labour_market_statistics
Consumer_price_indices
Purchasing_power_parities_Purchasing_Power_Parities
Purchasing_power_parities_Comparative_Price_Levels
Quarterly_national_accounts


In [90]:
series_list[0].index

Index(['2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1', '2020-Q2',
       '2020-Q3', '2020-Q4', '2021-Q1'],
      dtype='object', name='TIME_PERIOD')

In [99]:
data_response_btco = oecd.data(resource_id='PPPGDP', key='all?startTime=2018')
btco_series= data_response_btco.to_pandas()

In [100]:
stru_cos = getOECDStructure(dsname="PPPGDP")

Requesting URL http://stats.oecd.org/SDMX-JSON/dataflow/PPPGDP/all


In [101]:
list_ids= [item for item in stru_cos.get('structure').get('dimensions').get('observation') if item['keyPosition'] == 1]

In [104]:
dict_values={}
for did in list_ids[0].get('values'):
    dict_values[did['id']]=did['name'] 
dict_values["AUS"]

'Australia'

In [113]:
trial= btco_series.loc["PPP", "AUS"]

In [122]:
trial.loc['2018']

1.47

In [ ]:
 d = {"b": 1, "a": 0, "c": 2}

In [125]:
dict_new_series= {}
for t in trial.index:
    for q in quarters:
        new_index = [t + q for q in quarters]
        dict_new_series[t+q]= (trial.loc[t] / 4 ) 
      

In [127]:
pd.Series(dict_new_series)

2018-Q1    0.3675
2018-Q2    0.3675
2018-Q3    0.3675
2018-Q4    0.3675
2019-Q1    0.3675
2019-Q2    0.3675
2019-Q3    0.3675
2019-Q4    0.3675
2020-Q1    0.3650
2020-Q2    0.3650
2020-Q3    0.3650
2020-Q4    0.3650
dtype: float64

In [115]:
 s = pd.Series(data, index=index)

Index(['2018', '2019', '2020'], dtype='object')

In [33]:
btco_series.index.get_level_values(0

Index(['PIEAEN01', 'PIEAEN01', 'PIEAEN01', 'PIEAEN01', 'PIEAEN01', 'PIEAEN01',
       'PIEAEN01', 'PIEAEN01', 'PIEAEN01', 'PIEAEN01',
       ...
       'WPOTFD01', 'WPOTFD01', 'WPOTFD01', 'WPOTFD01', 'WPOTFD01', 'WPOTFD01',
       'WPOTFD01', 'WPOTFD01', 'WPOTFD01', 'WPOTFD01'],
      dtype='object', name='SUBJECT', length=101378)

In [36]:
economic_activities = [
    'PIEAMP02',
    'PIEAMP01',
    'PIEATI02',
    'PIEATI01',
    'PIEAMI02',
    'PIEAMI01',
    'PIEAFD02',
    'PIEAFD01',
    'PIEAEN02',
    'PIEAEN01',
]

In [40]:
 query_def= [economic_activities, 'AUS', 'GP', 'Q']

In [47]:
indicators = query_def[0]
for indi in indicators:
    query_def[0] = indi
    query= [str(item).replace('AUS', country) for item in query_def]
    query= [str(item).replace('Q', freq) for item in query]
    print(f" {query} -->")
        
           

 ['PIEAMP02', 'AUS', 'GP', 'Q'] -->
 ['PIEAMP01', 'AUS', 'GP', 'Q'] -->
 ['PIEATI02', 'AUS', 'GP', 'Q'] -->
 ['PIEATI01', 'AUS', 'GP', 'Q'] -->
 ['PIEAMI02', 'AUS', 'GP', 'Q'] -->
 ['PIEAMI01', 'AUS', 'GP', 'Q'] -->
 ['PIEAFD02', 'AUS', 'GP', 'Q'] -->
 ['PIEAFD01', 'AUS', 'GP', 'Q'] -->
 ['PIEAEN02', 'AUS', 'GP', 'Q'] -->
 ['PIEAEN01', 'AUS', 'GP', 'Q'] -->


In [57]:
ser = btco_series.loc[economic_activities, 'USA', 'GP']

In [58]:
ser.index

MultiIndex([('PIEAMP01', 'USA', 'GP', 'A',    '2018'),
            ('PIEAMP01', 'USA', 'GP', 'A',    '2019'),
            ('PIEAMP01', 'USA', 'GP', 'A',    '2020'),
            ('PIEAMP01', 'USA', 'GP', 'M', '2018-01'),
            ('PIEAMP01', 'USA', 'GP', 'M', '2018-02'),
            ('PIEAMP01', 'USA', 'GP', 'M', '2018-03'),
            ('PIEAMP01', 'USA', 'GP', 'M', '2018-04'),
            ('PIEAMP01', 'USA', 'GP', 'M', '2018-05'),
            ('PIEAMP01', 'USA', 'GP', 'M', '2018-06'),
            ('PIEAMP01', 'USA', 'GP', 'M', '2018-07'),
            ...
            ('PIEAEN01', 'USA', 'GP', 'Q', '2018-Q4'),
            ('PIEAEN01', 'USA', 'GP', 'Q', '2019-Q1'),
            ('PIEAEN01', 'USA', 'GP', 'Q', '2019-Q2'),
            ('PIEAEN01', 'USA', 'GP', 'Q', '2019-Q3'),
            ('PIEAEN01', 'USA', 'GP', 'Q', '2019-Q4'),
            ('PIEAEN01', 'USA', 'GP', 'Q', '2020-Q1'),
            ('PIEAEN01', 'USA', 'GP', 'Q', '2020-Q2'),
            ('PIEAEN01', 'USA', 'GP', 'Q', '2020-

In [59]:
index_unique_list = ser.index.get_level_values(0).unique()

In [60]:
index_unique_list

Index(['PIEAMP01', 'PIEAMI01', 'PIEAFD01', 'PIEAEN01'], dtype='object', name='SUBJECT')

In [51]:
for val in  index_unique_list:
    btco_series.loc['VAL', 'AUS', 'GP', 'Q']

Index(['PIEAMP01', 'PIEAFD01'], dtype='object', name='SUBJECT')

In [49]:
btco_series.loc['PIEAMP01', 'AUS', 'GP', 'Q']

TIME_PERIOD
2018-Q1    0.922509
2018-Q2    2.559415
2018-Q3    0.891266
2018-Q4    0.883392
2019-Q1    0.262697
2019-Q2    0.960699
2019-Q3    1.124568
2019-Q4    0.342173
2020-Q1    0.767263
2020-Q2   -1.099831
2020-Q3   -0.256630
2020-Q4    0.343053
2021-Q1    1.025641
Name: value, dtype: float64

In [47]:
ser.index.names

FrozenList(['SUBJECT', 'LOCATION', 'MEASURE', 'FREQUENCY', 'TIME_PERIOD'])

In [ ]:
btco_series.loc['PIEAMP01', 'AUS', 'GP', 'Q']

In [64]:
stru_cos = getOECDStructure(dsname="MEI_BOP6")

Requesting URL http://stats.oecd.org/SDMX-JSON/dataflow/MEI_BOP6/all


In [65]:
list_ids= [item for item in stru_cos.get('structure').get('dimensions').get('observation') if item['keyPosition'] == 0]